###This is lab 3

In [3]:
import requests,openpyxl,sqlite3


url = "http://kevincrook.com/utd/market_basket_training.txt"

r = requests.get(url)
type(r.content)
f = open("market_basket_training.txt","wb")
f.write(r.content)


url = "http://kevincrook.com/utd/market_basket_test.txt"

r = requests.get(url)
type(r.content)
f = open("market_basket_test","wb")
f.write(r.content)

f.close()

In [ ]:
import urllib2
for line in urllib2.urlopen("http://kevincrook.com/utd/market_basket_training.txt"):
    print line